In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np
tf.__version__

D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

'1.10.0'

In [2]:
CIFAR_DIR = 'E:/jupyter/cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [3]:
def load_data(filename):
    #read data form data file
    with open(filename,'rb') as f:
        data = pickle.load(f,encoding='bytes')
#         print(type(data))
#         print(data.keys())
        
#         print(type(data[b'labels']))
        return data[b'data'],data[b'labels']

#tesorflow.Dataset
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data/127.5 - 1  #归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples) #直接调用numoy的混排函数
        self._data = self._data[p]
        self._labels = self._labels[p]
    def next_batch(self,batch_size):
        #return batch_size examples as a batch
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator>self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data= self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels

train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d'%i) for i in range(1,6)]  
test_filenames = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [4]:
def residual_block(x,output_channel):
    input_channel = x.get_shape().as_list()[-1]
    if input_channel*2 == output_channel:
        increase_dim = True
        strides = (2,2)
    elif input_channel == output_channel:
        increase_dim = False
        strides = (1,1)
    else:
        raise Exception("input channel can't match output channel")
    conv1 = tf.layers.conv2d(x,
                            output_channel,
                            (3,3),
                            strides = strides,
                            padding = 'same',
                            activation = tf.nn.relu,
                            name = 'conv1')
    conv2 = tf.layers.conv2d(conv1,
                            output_channel,
                            (3,3),
                            strides = (1,1),
                            padding = 'same',
                            activation = tf.nn.relu,
                            name = 'conv2')
    if increase_dim:  #如果卷积的时候做了降采样，通道数会增加，所以恒等变化的x也应该降采样增加通道数
        #[None,image_width,image_hight,channel]
        pooled_x = tf.layers.average_pooling2d(x,
                                              (2,2),#kerner size
                                              (2,2),#strides   让图像变成原来的一半，但是并不改变通道数
                                              padding = 'valid')
        #补全缺失的通道数,保证padded_x和conv2大小相同
        padded_x = tf.pad(pooled_x,
                         [[0,0],
                         [0,0],
                         [0,0],
                         [input_channel//2,input_channel//2]])
    else:
        padded_x = x
    output_x = conv2 + padded_x
    return output_x

def res_net(x,num_residual_blocks,num_filter_base,class_num):
    #num_residual_blocks ，例如[3,4,6,3]即某个残差块的个数
    #num_subsampling = 4
    num_subsampling = len(num_residual_blocks)
    layers = []
    input_size = x.get_shape().as_list()[1:]
    with tf.variable_scope('conv0'):
        conv0 = tf.layers.conv2d(x,
                                num_filter_base,
                                (3,3),
                                strides = (1,1),
                                padding = 'same',
                                activation = tf.nn.relu,
                                name = 'conv0')
        layers.append(conv0)
    for sample_id in range(num_subsampling):
        for i in range(num_residual_blocks[sample_id]):
            with tf.variable_scope("conv%d_%d"%(sample_id,i)):
                conv = residual_block(
                layers[-1],
                num_filter_base*(2**sample_id))
                layers.append(conv)
    multiplier = 2**(num_subsampling-1)
    assert layers[-1].get_shape().as_list()[1:] == [input_size[0]/multiplier,input_size[1]/multiplier,num_filter_base*multiplier]
    with tf.variable_scope('fc'):
        #layer[-1].shape:[None,width,height,channel]
        # kernel_size:image_width,image_height
        global_pool = tf.reduce_mean(layers[-1],[1,2])
        logits = tf.layers.dense(global_pool,class_num)
        layers.append(logits)
    return layers[-1]
        
    

x = tf.placeholder(tf.float32,[None,3072])
y = tf.placeholder(tf.int64,[None])

x_image = tf.reshape(x,[-1,3,32,32])
x_image = tf.transpose(x_image,perm=[0,2,3,1])




y_ = res_net(x_image,[2,3,2],32,10)  #调用残差

#调用api去做交叉熵损失
loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
#y_ ->softmax
#y -> one-hot
#loss = ylogy_

#index
predict = tf.argmax(y_,1)
correct_prediction = tf.equal(predict,y)
acc = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdadeltaOptimizer(1e-3).minimize(loss)


In [5]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000

test_steps = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        loss_val,acc_val,_ = sess.run(
            [loss,acc,train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels
            }
        )
        if i%500 ==0:
            print('[Train] Steps: %d, loss: %4.5f, acc: %4.5f'%(i,loss_val,acc_val))
        
        if (i+1)%5000 == 0:
            test_data = CifarData(test_filenames,False)
            for j in range(test_steps):
                test_batch_data,test_batch_labels = test_data.next_batch(batch_size)
                all_test_acc_val = []
                test_acc_val = sess.run(
                    [acc],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    }
                )
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1,test_acc))
        

[Train] Steps: 0, loss: 2.26080, acc: 0.30000
[Train] Steps: 500, loss: 2.31767, acc: 0.15000
